# Computation Intelligence for Optimization | Sports League Optimization

`Group AM`
- Eduardo Mendes, 20240850
- Helena Duarte, 20240530
- João Freire, 20240528
- Mariana Sousa, 20240516

<div class="alert alert-block alert-info">

# Table of Contents
    
[1. Import Libraries](#1)<br>

[2. Load data](#2)<br>

<a class="anchor" id="1">

# 1. Import Libraries
    
</a>

In [1]:
import os
import pandas as pd

from copy import deepcopy
from random import random, sample, choice, randint
import copy

<a class="anchor" id="2">

# 2. Load data
    
</a>

In [2]:
data_dir= os.path.join(os.getcwd(), 'players(in).csv')

df = pd.read_csv(data_dir, index_col=0)
df.head()

,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


In [3]:
players = df.to_dict(orient="records")
players

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

<a class="anchor" id="3">

# 3. Problem Definiton
    
</a>

In a fantasy sports league, the objective is to assign players to teams in a way that ensures
a balanced distribution of talent while staying within salary caps.

1) Each player is defined by the following attributes:
* Skill rating: Represents the player's ability.
* Cost: The player's salary.
* Position (One of four roles) : Goalkeeper (GK), Defender (DEF), Midfielder (MID), or Forward (FWD).

A solution is a complete league configuration, specifying the team assignment for each player. These are the constraints that must be verified in every solution of the search space (no object is considered a solution if it doesn’t comply with these):
* Each team must consist of: 1 Goalkeeper, 2 Defenders, 2 Midfielders and 2
Forwards.
* Each player is assigned to exactly one team.

*Impossible Configurations*: Teams that do not follow this exact structure (e.g., a team with 2 goalkeepers, or a team where the same defender is assigned twice) are not part of the search space and are not considered solutions. It is forbidden to generate such an arrangement during evolution.

Besides that, each team should not exceed a 750€ million total budget. If it does, it is not a valid solution and the fitness value should reflect that.

The `objective` is to create a balanced league that complies with the constraints. 
A balanced league a is a league where the average skill rating of the players is roughly the same among the teams. 
This can be measured by the standard deviation of the average skill rating of the teams.

You can find a dataset of players with their names, position, skill rating and salary (in million €).
These players should be distributed across 5 teams of 7 players each.

In [4]:
def average_skill_rating(team):
    count = 0
    total_skill = 0

    for player in team:
        total_skill += player["Skill"]
        count += 1

    average = total_skill/count
    return average
def has_repeated_players(solution):

    players = []
    for team in solution: #5 teams
        for player in team: #7 players for each team
            if player['Name'] in players:
                return True
            players.append(player['Name'])
    
    return False

# 4. Fitness Function

<hr>

### THE ONE

In [ ]:
import random
import copy
    
class League():
    def __init__(self, name, players):
        self.name = name
        self.players = players
        self.gk = []
        self.defenders = []
        self.midfielders = []
        self.forwards = []
        self.total_skill = 0
        self.average_skill = 0
        self.total_cost = 0

        for player in players:
            self.add_player(player)

    @staticmethod
    def random_teams(player_pool, num_teams=5, max_budget=750):
        positions = {
            'GK': [p for p in player_pool if p['Position'] == 'GK'],
            'DEF': [p for p in player_pool if p['Position'] == 'DEF'],
            'MID': [p for p in player_pool if p['Position'] == 'MID'],
            'FWD': [p for p in player_pool if p['Position'] == 'FWD']
        }
        
        used_players = set()
        teams = []

        for i in range(num_teams):
            number_tries = 0
            max_tries = 1000
            team = None

            while number_tries < max_tries:
                number_tries += 1
                try:
                    gk = random.choice([p for p in positions['GK'] if p['Name'] not in used_players])
                    def_players = random.sample([p for p in positions['DEF'] if p['Name'] not in used_players], 2)
                    mid_players = random.sample([p for p in positions['MID'] if p['Name'] not in used_players], 2)
                    fwd_players = random.sample([p for p in positions['FWD'] if p['Name'] not in used_players], 2)

                    team = [gk] + def_players + mid_players + fwd_players 
                    total_cost = sum(p["Salary (€M)"] for p in team)
                    total_skill = sum(p["Skill"] for p in team)
                    
                    if total_cost <= max_budget:
                         break

                except ValueError:
                    break

            if team is None:
                raise ValueError(f"Could not form a valid team {i+1} within budget after {max_tries} tries")

            used_players.update([p['Name'] for p in team])
            for p in team:
                positions[p['Position']].remove(p)
            teams.append((f"Team {i+1}", team))
        
        return teams

    @staticmethod
    def initialize_population(player_pool, num_leagues=3, num_teams=5, max_budget=750):
        all_solutions = []
        for league_num in range(num_leagues):
            # Generate ALL teams for the league at once
            league_teams = League.random_teams(
                copy.deepcopy(player_pool), 
                num_teams=num_teams,  # Generate 5 teams in one call
                max_budget=max_budget
        )
        
            print(f"League {league_num + 1}:")
            for team_idx, (team_name, team) in enumerate(league_teams, 1):
                total_cost = sum(p["Salary (€M)"] for p in team)
                avg_skill = sum(p["Skill"] for p in team) / len(team)
                print(f"  Team {team_idx}: {len(team)} players, Total Cost: {total_cost}M, Average Skill: {avg_skill:.2f}")
        
            all_solutions.append(league_teams)
    
        return all_solutions
        

    def valid_budget(self, budget):
        return self.calculate_total_cost() <= budget
    
    def calculate_total_skill(self):
        self.total_skill = sum(player["Skill"] for player in self.players)
        return self.total_skill
    
    def calculate_total_cost(self):
        self.total_cost = sum(player["Salary (€M)"] for player in self.players)
        return self.total_cost
    
    def calculate_avg_skill(self):
        self.average_skill = self.calculate_total_skill() / len(self.players) if self.players else 0
        return self.average_skill
    
    def fitness(self, budget):
        if not self.valid_budget(budget):
            return -100000
        else:
            cost_penalty = max(0, self.calculate_total_cost() - 100) * 5
            return self.calculate_total_skill() - cost_penalty
    
    # Magic methods
    def __str__(self):
        return f"Team: {self.name}, Players: {len(self.players)}, Average Skill: {self.average_skill}, Total Cost: {self.total_cost}"
    def __repr__(self):
        return f"Team({self.name}, {self.players})"
    def __len__(self):
        return len(self.players)
    def __getitem__(self, index):
        return self.players[index]
    def __setitem__(self, index, value):
        self.players[index] = value
    def __delitem__(self, index):
        del self.players[index]
    def __contains__(self, item):
        return item in self.players
    def __iter__(self):
        return iter(self.players)

    def add_player(self, player):
        pos = player['Position']
        if pos == 'GK':
            self.gk.append(player)
        elif pos == 'DEF':
            self.defenders.append(player)
        elif pos == 'MID':
            self.midfielders.append(player)
        elif pos == 'FWD':
            self.forwards.append(player)
        self.players.append(player)

In [20]:
solution_leagues = League.initialize_population(players, num_leagues = 3, num_teams=5, max_budget=750)
solution_leagues = population

League 1:
  Team 1: 7 players, Total Cost: 680M, Average Skill: 86.00
  Team 2: 7 players, Total Cost: 667M, Average Skill: 86.00
  Team 3: 7 players, Total Cost: 722M, Average Skill: 87.86
  Team 4: 7 players, Total Cost: 660M, Average Skill: 85.00
  Team 5: 7 players, Total Cost: 695M, Average Skill: 87.14
League 2:
  Team 1: 7 players, Total Cost: 657M, Average Skill: 85.43
  Team 2: 7 players, Total Cost: 700M, Average Skill: 87.43
  Team 3: 7 players, Total Cost: 640M, Average Skill: 84.86
  Team 4: 7 players, Total Cost: 702M, Average Skill: 87.43
  Team 5: 7 players, Total Cost: 725M, Average Skill: 86.86
League 3:
  Team 1: 7 players, Total Cost: 665M, Average Skill: 86.43
  Team 2: 7 players, Total Cost: 667M, Average Skill: 86.00
  Team 3: 7 players, Total Cost: 642M, Average Skill: 84.29
  Team 4: 7 players, Total Cost: 705M, Average Skill: 86.57
  Team 5: 7 players, Total Cost: 745M, Average Skill: 88.71


In [22]:
for league_num, league in enumerate(solution_leagues, 1):
    print(f"League {league_num}:")
    for team_name, team_members in league:
        print(f"  {team_name}:")
        for player in team_members:
            print(f"    {player['Name']} ({player['Position']}, Skill: {player['Skill']}, Salary: {player['Salary (€M)']}M)")


League 1:
  Team 1:
    Blake Henderson (GK, Skill: 87, Salary: 95M)
    Daniel Foster (DEF, Skill: 90, Salary: 110M)
    Mason Reed (DEF, Skill: 82, Salary: 75M)
    Dominic Bell (MID, Skill: 86, Salary: 95M)
    Nathan Wright (MID, Skill: 92, Salary: 120M)
    Elijah Sanders (FWD, Skill: 93, Salary: 140M)
    Tyler Jenkins (FWD, Skill: 80, Salary: 70M)
  Team 2:
    Ryan Mitchell (GK, Skill: 83, Salary: 85M)
    Caleb Fisher (DEF, Skill: 84, Salary: 85M)
    Logan Brooks (DEF, Skill: 86, Salary: 95M)
    Ashton Phillips (MID, Skill: 90, Salary: 110M)
    Hunter Cooper (MID, Skill: 83, Salary: 85M)
    Xavier Bryant (FWD, Skill: 90, Salary: 120M)
    Colton Gray (FWD, Skill: 91, Salary: 125M)
  Team 3:
    Chris Thompson (GK, Skill: 80, Salary: 80M)
    Maxwell Flores (DEF, Skill: 81, Salary: 72M)
    Owen Parker (DEF, Skill: 88, Salary: 100M)
    Connor Hayes (MID, Skill: 89, Salary: 105M)
    Gavin Richardson (MID, Skill: 87, Salary: 95M)
    Julian Scott (FWD, Skill: 92, Salary: 13

<hr>

In [5]:
def initialize_population(player_pool, population_size=100, max_budget=100):
    population = []
    position_groups = {
        'GK': [p for p in player_pool if p['Position'] == 'GK'],
        'DEF': [p for p in player_pool if p['Position'] == 'DEF'],
        'MID': [p for p in player_pool if p['Position'] == 'MID'],
        'FWD': [p for p in player_pool if p['Position'] == 'FWD']
    }
    
    for _ in range(population_size):
        while True:
            team_players = [
                random.choice(position_groups['GK']),
                *random.sample(position_groups['DEF'], k=2),
                *random.sample(position_groups['MID'], k=2),
                *random.sample(position_groups['FWD'], k=2)
            ]
            
            # Add optional players (up to 11 total)
            remaining_spots = 11 - len(team_players)
            for _ in range(remaining_spots):
                position = random.choice(['DEF', 'MID', 'FWD'])
                team_players.append(random.choice(position_groups[position]))
                
            # Check budget constraint
            total_cost = sum(p['Cost'] for p in team_players)
            if total_cost <= max_budget:
                population.append(Team(f"Team_{len(population)+1}", team_players))
                break
                
    return population


In [6]:
# Função para gerar uma solução com 5 equipas válidas e sem jogadores repetidos
import random
import copy
def generate_solution(player_pool, max_budget=750):
    positions = {
        'GK': [p for p in player_pool if p['Position'] == 'GK'],
        'DEF': [p for p in player_pool if p['Position'] == 'DEF'],
        'MID': [p for p in player_pool if p['Position'] == 'MID'],
        'FWD': [p for p in player_pool if p['Position'] == 'FWD']
    }
    used_names = set()
    teams = []

    for i in range(5):
        while True:
            gk = random.choice([p for p in positions['GK'] if p['Name'] not in used_names])
            def_players = random.sample([p for p in positions['DEF'] if p['Name'] not in used_names], 2)
            mid_players = random.sample([p for p in positions['MID'] if p['Name'] not in used_names], 2)
            fwd_players = random.sample([p for p in positions['FWD'] if p['Name'] not in used_names], 2)

            team = [gk] + def_players + mid_players + fwd_players
            total_cost = sum(p['Salary (€M)'] for p in team)

            if total_cost <= max_budget:
                used_names.update(p["Name"] for p in team)
                for p in team:
                    positions[p["Position"]].remove(p)
                teams.append(League(f"Team {i+1}", team))
                print(team)
                break
            used_names.update(p["Name"] for p in team)
            for p in team:
                positions[p["Position"]].remove(p)
            teams.append(League(f"Team {i+1}", team))
           
    return teams

# Inicializar uma população
def initialize_population(player_pool, population_size=2, max_budget=750):
    population = []
    for _ in range(population_size):
        print("New Population")
        solution = generate_solution(copy.deepcopy(player_pool), max_budget)
        population.append(solution)
    return population

# Gerar população e mostrar uma solução
population = initialize_population(players, population_size=10)
solution = population
solution_to_show = population[0]

df_pop = pd.DataFrame([{
    "Team": team.name,
    "Valid": team.valid_formation(),
    'Salary (€M)': team.calculate_total_cost(),
    "Avg Skill": team.calculate_avg_skill()
} for team in solution_to_show])




New Population
[{'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80}, {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70}, {'Name': 'Maxwell Flores', 'Position': 'DEF', 'Skill': 81, 'Salary (€M)': 72}, {'Name': 'Ashton Phillips', 'Position': 'MID', 'Skill': 90, 'Salary (€M)': 110}, {'Name': 'Dominic Bell', 'Position': 'MID', 'Skill': 86, 'Salary (€M)': 95}, {'Name': 'Xavier Bryant', 'Position': 'FWD', 'Skill': 90, 'Salary (€M)': 120}, {'Name': 'Zachary Nelson', 'Position': 'FWD', 'Skill': 86, 'Salary (€M)': 92}]
[{'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85}, {'Name': 'Jaxon Griffin', 'Position': 'DEF', 'Skill': 79, 'Salary (€M)': 65}, {'Name': 'Brayden Hughes', 'Position': 'DEF', 'Skill': 87, 'Salary (€M)': 100}, {'Name': 'Connor Hayes', 'Position': 'MID', 'Skill': 89, 'Salary (€M)': 105}, {'Name': 'Austin Torres', 'Position': 'MID', 'Skill': 82, 'Salary (€M)': 80}, {'Name': 'Sebastian Perry', 'Position': 'F

IndexError: Cannot choose from an empty sequence

In [ ]:
# Função para gerar uma solução com 5 equipas válidas e sem jogadores repetidos

def generate_solution(player_pool, max_budget=750):
    positions = {
        'GK': [p for p in player_pool if p['Position'] == 'GK'],
        'DEF': [p for p in player_pool if p['Position'] == 'DEF'],
        'MID': [p for p in player_pool if p['Position'] == 'MID'],
        'FWD': [p for p in player_pool if p['Position'] == 'FWD']
    }
    used_names = set()
    teams = []

    for i in range(5):
        number_tries = 0
        max_tries = 100
        team = None

        while number_tries < max_tries:
            number_tries += 1
            try:
                gk = random.choice([p for p in positions['GK'] if p['Name'] not in used_names])
                def_players = random.sample([p for p in positions['DEF'] if p['Name'] not in used_names], 2)
                mid_players = random.sample([p for p in positions['MID'] if p['Name'] not in used_names], 2)
                fwd_players = random.sample([p for p in positions['FWD'] if p['Name'] not in used_names], 2)

                team_candidate = [gk] + def_players + mid_players + fwd_players
                total_cost = sum(p['Salary (€M)'] for p in team_candidate)

                if total_cost <= max_budget:
                    team = team_candidate
                    print(f"⚠️ Equipa {i+1} criada DENTRO do budget: {sum(p['Salary (€M)'] for p in team)}M")
                    break
            except Exception:
                break  # jogadores insuficientes

    # se não conseguiu dentro do budget, aceita a última combinação possível
        if team is None:
            try:
                gk = random.choice([p for p in positions['GK'] if p['Name'] not in used_names])
                def_players = random.sample([p for p in positions['DEF'] if p['Name'] not in used_names], 2)
                mid_players = random.sample([p for p in positions['MID'] if p['Name'] not in used_names], 2)
                fwd_players = random.sample([p for p in positions['FWD'] if p['Name'] not in used_names], 2)

                team = [gk] + def_players + mid_players + fwd_players
                print(f"⚠️ Equipa {i+1} criada FORA do budget: {sum(p['Salary (€M)'] for p in team)}M")
            except Exception:
                print(f"❌ Não foi possível formar a equipa {i+1}")
                break

        used_names.update(p["Name"] for p in team)
        for p in team:
            positions[p["Position"]].remove(p)
        teams.append(League(f"Team {i+1}", team))
    return teams if len(teams) == 5 else None

# Inicializar uma população
def initialize_population(player_pool, population_size=5, max_budget=750):
    population = []
    for _ in range(population_size):
        print("New Population")
        solution = generate_solution(copy.deepcopy(player_pool), max_budget)
        if solution is not None and len(solution) == 5:
            population.append(solution)
        else:
            print("❌ Solução incompleta descartada.")
    return population

# Gerar população e mostrar uma solução
population = initialize_population(players, population_size=5)

solution = population

New Population
⚠️ Equipa 1 criada DENTRO do budget: 637M
⚠️ Equipa 2 criada DENTRO do budget: 672M
⚠️ Equipa 3 criada DENTRO do budget: 745M
⚠️ Equipa 4 criada DENTRO do budget: 575M
⚠️ Equipa 5 criada FORA do budget: 795M
New Population
⚠️ Equipa 1 criada DENTRO do budget: 687M
⚠️ Equipa 2 criada DENTRO do budget: 625M
⚠️ Equipa 3 criada DENTRO do budget: 647M
⚠️ Equipa 4 criada DENTRO do budget: 720M
⚠️ Equipa 5 criada DENTRO do budget: 745M
New Population
⚠️ Equipa 1 criada DENTRO do budget: 640M
⚠️ Equipa 2 criada DENTRO do budget: 670M
⚠️ Equipa 3 criada DENTRO do budget: 737M
⚠️ Equipa 4 criada DENTRO do budget: 640M
⚠️ Equipa 5 criada DENTRO do budget: 737M
New Population
⚠️ Equipa 1 criada DENTRO do budget: 617M
⚠️ Equipa 2 criada DENTRO do budget: 735M
⚠️ Equipa 3 criada DENTRO do budget: 705M
⚠️ Equipa 4 criada DENTRO do budget: 717M
⚠️ Equipa 5 criada DENTRO do budget: 650M
New Population
⚠️ Equipa 1 criada DENTRO do budget: 727M
⚠️ Equipa 2 criada DENTRO do budget: 702M
⚠️ 

In [47]:
import numpy as np

def fitness_solution(teams, budget=750):
    avg_skills = []
    penalty = 0

    for team in teams:
        if not team.valid_formation():
            return -10000
        #if has_repeated_players(team):
            #return -100000
        if not team.valid_budget(budget):
            penalty = (team.calculate_total_cost() - budget) * 5
        avg_skills.append(team.calculate_avg_skill())

    std_dev = np.std(avg_skills) 
    print(f"Average Skill :{std_dev}") # equilíbrio entre as equipas
    return -std_dev + (-penalty)


In [48]:
for i, solution in enumerate(population):
    print(f"Solution {i+1} → Fitness: {fitness_solution(solution)}")


Average Skill :2.4898159918896408
Solution 1 → Fitness: -227.48981599188963
Average Skill :1.4774495398683285
Solution 2 → Fitness: -1.4774495398683285
Average Skill :1.4211234417652434
Solution 3 → Fitness: -1.4211234417652434
Average Skill :1.0705710854397292
Solution 4 → Fitness: -1.0705710854397292
Average Skill :0.8637837975903412
Solution 5 → Fitness: -0.8637837975903412
